In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import platform
import utils

In [ ]:
### MODEL PARAMETERS ###

optimizer_func = "adam"
loss_func = "binary_crossentropy"
epochs_num = 3

model_name = "pneumonia_detector.model"

### CACHE FILENAMES ###
train_normal_cache = "training_normal.pickle"
train_pneumonia_cache = "training_pneumonia.pickle"
val_normal_cache = "validation_normal.pickle"
val_pneumonia_cache = "validation_pneumonia.pickle"
test_normal_cache = "testing_normal.pickle"
test_pneumonia_cache = "testing_pneumonia.pickle"

In [ ]:
### SETTINGS ###

pdiv = "/" if platform.system() == "Windows" else "\\"

# Data paths
path_train_normal = ".{}chest_xray{}train{}NORMAL{}".format(pdiv, pdiv, pdiv, pdiv)
path_train_pneumonia = ".{}chest_xray{}train{}PNEUMONIA{}".format(pdiv, pdiv, pdiv, pdiv)
path_val_normal = ".{}chest_xray{}val{}NORMAL{}".format(pdiv, pdiv, pdiv, pdiv)
path_val_pneumonia = ".{}chest_xray{}val{}PNEUMONIA{}".format(pdiv, pdiv, pdiv, pdiv)
path_test_normal = ".{}chest_xray{}test{}NORMAL{}".format(pdiv, pdiv, pdiv, pdiv)
path_test_pneumonia = ".{}chest_xray{}test{}PNEUMONIA{}".format(pdiv, pdiv, pdiv, pdiv)

# Labels
normal_label = "NORMAL"
pneumonia_label = "PNEUMONIA"

labels = sorted([normal_label, pneumonia_label])

In [ ]:
### MODEL CREATION ###

model = tf.keras.models.Sequential()

# Input layer - Flatten the image (each pixel is the input)
model.add(tf.keras.layers.Flatten())

# Hidden layer
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))

# Output layer - NORMAL or PNEUMONIA
model.add(tf.keras.layers.Dense(2, activation=tf.nn.softmax))

# Define optimiser, loss function, and metrics to track
model.compile(optimizer=optimizer_func, loss=loss_func, metrics=['accuracy'])

In [ ]:
### LOAD TRAINING DATA AND LABELS ###

data_train_normal, label_train_normal = utils.load_data(path_train_normal, train_normal_cache, normal_label)
data_train_pneumonia, label_train_pneumonia = utils.load_data(path_train_pneumonia, train_pneumonia_cache, pneumonia_label)

# Caching, disabled for now
# utils.cache(data_train_normal, train_normal_cache)
# utils.cache(data_train_pneumonia, train_pneumonia_cache)

# Combine training images and labels
data_train = np.concatenate((data_train_normal, data_train_pneumonia), axis=0)
label_train = np.asarray(label_train_normal + label_train_pneumonia)

del data_train_normal
del data_train_pneumonia

In [ ]:
### IMAGE PROCESSING ###

# TODO: image cropping

# TODO: image normalisation
# img_training_normal = tf.keras.utils.normalize(img_training_normal, axis=0)

# TODO: image resizing

In [ ]:
# Model training
model.fit(data_train, label_train, epochs=epochs_num)

In [ ]:
del data_train

In [ ]:
### LOAD VALIDATION DATA AND LABELS ###

data_val_normal, label_val_normal = utils.load_data(path_val_normal, val_normal_cache, normal_label)
data_val_pneumonia, label_val_pneumonia = utils.load_data(path_val_pneumonia, val_pneumonia_cache, pneumonia_label)

# Caching, disabled for now
# utils.cache(data_val_normal, val_normal_cache)
# utils.cache(data_val_pneumonia, val_pneumonia_cache)

# Combine validation images and labels
data_val = np.concatenate((data_val_normal, data_val_pneumonia), axis=0)
label_val = np.asarray(label_val_normal + label_val_pneumonia)

del data_val_normal
del data_val_pneumonia

In [ ]:
### LOAD TESTING DATA AND LABELS ###

data_test_normal, label_test_normal = utils.load_data(path_test_normal, test_normal_cache, normal_label)
data_test_pneumonia, label_test_pneumonia = utils.load_path(path_test_pneumonia, test_pneumonia_cache, pneumonia_label)

# Caching, disabled for now
# utils.cache(data_test_normal, test_normal_cache)
# utils.cache(data_test_pneumonia, test_pneumonia_cache)

# Combine testing images and labels
data_test = np.concatenate((data_test_normal, data_test_pneumonia), axis=0)
label_test = np.asarray(label_test_normal, label_test_pneumonia)

del data_test_normal
del data_val_pneumonia

In [ ]:
# Validation and testing
val_loss, val_accuracy = model.evaluate(data_val, label_val)
test_loss, test_accuracy = model.evaluate(data_test, label_test)

In [ ]:
del data_val
del data_test

In [ ]:
# Save the model for usage
model.save(model_name)

In [ ]:
# Load model
# new_model = tf.keras.models.load_model(model_name)

In [ ]:
# to_test = np.asarray([test1, test2, ...])
# predictions = new_model.predict(to_test)
# TODO: convert to class labels and probability